# Estimación de dirección de arribo de fuentes sonoras 

In [1]:
import numpy as np
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from tdoa_py import correlation, simulation_room

In [2]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=0#gid=0")
sheet = spreadsheet.sheet1

# Leer datos
data = sheet.get_all_records()
df_simulations = pd.DataFrame(data)

df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1)
df_simulations.head() # Verificar el resultado



,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,src_az_deg,src_z,theta_1,theta_2,theta_3,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim
0,sim_cc5,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,5,1.2,,,,,,,,"(100, 100, 100)"
1,sim_cc10,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,10,1.2,,,,,,,,"(100, 100, 100)"
2,sim_cc15,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,15,1.2,,,,,,,,"(100, 100, 100)"
3,sim_cc20,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,20,1.2,,,,,,,,"(100, 100, 100)"
4,sim_cc25,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,25,1.2,,,,,,,,"(100, 100, 100)"


In [3]:
# df_simulations = pd.read_csv('simulations.csv')
# df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1)
# df_simulations.head() # Verificar el resultado

In [4]:
# Iterar sobre el DataFrame con indexación correcta
for idx, sim in df_simulations.iterrows():
    src_az_deg, mic_signals = simulation_room.sim_room_Nmics(
        wav_path=f'audios/anechoic/{sim["audio"]}',
        out_dir=f'audios/output/{sim["sim_name"]}',
        fs=48000,
        room_dim=sim["room_dim"],
        rt60=sim["rt60"],
        snr_db=sim["snr_db"],
        n_mics=sim["n_mics"],
        mic_d=sim["mic_d"],
        mic_z=sim["mic_z"],
        mic_directivity=sim["mic_directivity"],
        src_dist=sim["src_dist"],
        src_az_deg=sim["src_az_deg"],
        src_z=sim["src_z"],
        save_audio=False
    )
    
    avg_angle_deg, avg_tdoa, hemi_avgs, tdoas, angles = correlation.estimate_doa(
        signal_input=mic_signals,
        mic_d=sim["mic_d"],
        method=sim["method"],
        fs=48000
    )

    # Guardar resultados en cada fila individual
    df_simulations.at[idx, 'avg_angle_deg'] = avg_angle_deg
    df_simulations.at[idx, 'avg_tdoa'] = avg_tdoa
    df_simulations.at[idx, 'hemi_1'] = hemi_avgs['H1']
    df_simulations.at[idx, 'hemi_2'] = hemi_avgs['H2']
    df_simulations.at[idx, 'theta_1'] = angles[0]
    df_simulations.at[idx, 'theta_2'] = angles[1]
    df_simulations.at[idx, 'theta_3'] = angles[2]

# Convertir el DataFrame a una lista de listas para Google Sheets
df_simulations["room_dim"] = df_simulations["room_dim"].apply(lambda x: ",".join(map(str, x)))
data = [df_simulations.columns.values.tolist()] + df_simulations.values.tolist()

# Escribir en la hoja de cálculo
sheet.update(data)
print("DataFrame guardado correctamente en Google Sheets.")

df_simulations



{'H1': [np.float64(0.0), np.float64(0.0)], 'H2': [np.float64(180.0)]}
{'H1': [np.float64(15.271729393963701), np.float64(0.0)], 'H2': [np.float64(180.0)]}
{'H1': [np.float64(15.271729393963701), np.float64(21.726983901452964)], 'H2': [np.float64(201.72698390145297)]}
{'H1': [np.float64(21.726983901452964), np.float64(21.726983901452964)], 'H2': [np.float64(201.72698390145297)]}
{'H1': [np.float64(26.718129966783323), np.float64(21.726983901452964)], 'H2': [np.float64(201.72698390145297)]}
{'H1': [np.float64(30.962968709327708), np.float64(30.962968709327708)], 'H2': [np.float64(210.9629687093277)]}
{'H1': [np.float64(34.73754414764358), np.float64(38.182877412288086)], 'H2': [np.float64(218.18287741228806)]}
{'H1': [np.float64(41.38254514217342), np.float64(38.182877412288086)], 'H2': [np.float64(218.18287741228806)]}
{'H1': [np.float64(44.39093778146574), np.float64(44.39093778146574)], 'H2': [np.float64(224.3909377814657)]}
{'H1': [np.float64(49.97476045263578), np.float64(49.9747604

,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,src_az_deg,src_z,theta_1,theta_2,theta_3,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim
0,sim_cc5,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,5,1.2,0.0,0.0,180.0,0.0,0.000194,0.0,180.0,"100,100,100"
1,sim_cc10,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,10,1.2,15.271729,0.0,180.0,7.635865,0.000188,7.635865,180.0,"100,100,100"
2,sim_cc15,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,15,1.2,15.271729,21.726984,201.726984,18.499357,0.000187,18.499357,201.726984,"100,100,100"
3,sim_cc20,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,20,1.2,21.726984,21.726984,201.726984,21.726984,0.000181,21.726984,201.726984,"100,100,100"
4,sim_cc25,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,25,1.2,26.71813,21.726984,201.726984,24.222557,0.000174,24.222557,201.726984,"100,100,100"
5,sim_cc30,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,30,1.2,30.962969,30.962969,210.962969,30.962969,0.000167,30.962969,210.962969,"100,100,100"
6,sim_cc35,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,35,1.2,34.737544,38.182877,218.182877,36.460211,0.00016,36.460211,218.182877,"100,100,100"
7,sim_cc40,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,40,1.2,41.382545,38.182877,218.182877,39.782711,0.000146,39.782711,218.182877,"100,100,100"
8,sim_cc45,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,45,1.2,44.390938,44.390938,224.390938,44.390938,0.000139,44.390938,224.390938,"100,100,100"
9,sim_cc50,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,50,1.2,49.97476,49.97476,229.97476,49.97476,0.000125,49.97476,229.97476,"100,100,100"
